In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from eo_tools.S1.core import S1IWSwath
from eo_tools.S1.core import align, coregister
from eo_tools.util import show_cog
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
import numpy as np

from folium import LayerControl
from eo_tools.S1.util import boxcar, presum
import xarray as xr

import logging

log = logging.getLogger(__name__)

In [ ]:
data_dir = "/data/S1"
master_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230903T183344_20230903T183412_050167_0609B4_100E.SAFE"
slave_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230915T183345_20230915T183413_050342_060F9F_85A4.SAFE"
iw = 1
pol = "vv"
up = 2

In [ ]:
ifgs = []
lut = []
dems = []

prm = S1IWSwath(master_dir, iw=iw, pol=pol)
sec = S1IWSwath(slave_dir, iw=iw, pol=pol)
overlap = np.round(prm.compute_burst_overlap(2)).astype(int)

boxsiz = 5

def avg_ampl(arr, boxsiz):
    return np.sqrt(boxcar(np.nan_to_num(abs(arr)**2, 1e-10), boxsiz, boxsiz))

for burst_idx in range(1, 3):
    log.info(f"---- Processing burst {burst_idx} ----")

    # compute geocoding LUTs for master and slave bursts
    file_dem = prm.fetch_dem_burst(burst_idx, dir_dem="/data/tmp", force_download=False)
    az_p2g, rg_p2g = prm.geocode_burst(file_dem, burst_idx=burst_idx, dem_upsampling=up)
    az_s2g, rg_s2g = sec.geocode_burst(file_dem, burst_idx=burst_idx, dem_upsampling=up)

    # read primary and secondary burst raster
    arr_p = prm.read_burst(burst_idx, True)
    arr_s = sec.read_burst(burst_idx, True)

    # deramp secondary
    pdb_s = sec.deramp_burst(burst_idx)
    arr_s_de = arr_s * np.exp(1j * pdb_s).astype(np.complex64)

    # project slave LUT into master grid
    az_s2p, rg_s2p = coregister(arr_p, az_p2g, rg_p2g, az_s2g, rg_s2g)

    # warp raster slave and deramping phase
    arr_s2p = align(arr_p, arr_s_de, az_s2p, rg_s2p, order=3)
    pdb_s2p = align(arr_p, pdb_s, az_s2p, rg_s2p, order=3)

    # reramp slave
    arr_s2p = arr_s2p * np.exp(-1j * pdb_s2p).astype(np.complex64)

    # compute topographic phases
    rg_p = np.zeros(arr_s.shape[0])[:, None] + np.arange(0, arr_s.shape[1])
    pht_p = prm.phi_topo(rg_p).reshape(*arr_p.shape)
    pht_s = sec.phi_topo(rg_s2p.ravel()).reshape(*arr_p.shape)
    pha_topo = np.exp(-1j * (pht_p - pht_s)).astype(np.complex64)

    # interferogram without topographic phase
    ifg = (arr_s2p * arr_p.conj() * pha_topo).conj()

    # normalize complex coherences  
    pows = avg_ampl(arr_p, boxsiz) * avg_ampl(arr_s2p, boxsiz)
    ifgs.append(boxcar(np.nan_to_num(ifg), boxsiz, boxsiz) / pows)
    lut.append((az_p2g, rg_p2g))
    dems.append(file_dem)

In [ ]:
from eo_tools.S1_dev import fast_esd
fast_esd(ifgs, overlap)

In [ ]:
from eo_tools.S1_dev import stitch_bursts
img = stitch_bursts(ifgs, overlap)

In [ ]:
from eo_tools.S1_dev import resample
from eo_tools.util import palette_phi
from folium import Map
import rioxarray as riox
from rioxarray.merge import merge_arrays

off = 0
H = int(overlap / 2)
phi_out = presum(np.nan_to_num(img), 2, 8)
naz = ifgs[0].shape[0]
list_ifg = []
for i in range(len(ifgs)):
    az_mst, rg_mst = lut[i]
    file_dem = dems[i]
    cnd = (az_mst >= H - 4) & (az_mst < naz - H + 4)
    az_mst2 = az_mst.copy()
    rg_mst2 = rg_mst.copy()
    az_mst2[~cnd] = np.nan
    rg_mst2[~cnd] = np.nan

    file_ifg = f"/data/res/test_remap_burst_{i}_ifg.tif"

    # does the job but not very elegant
    if i == 0:
        off2 = off
    else:
        off2 = off - H
    resample(
        phi_out,
        file_dem,
        file_ifg,
        (az_mst2 + off2) / 2,
        (rg_mst2) / 8,
        order=3,
    )
    if i == 0:
        off += naz - H
    else:
        off += naz - 2 * H

    list_ifg.append(riox.open_rasterio(file_ifg))

merged_ifg = merge_arrays(list_ifg)
merged_ifg.rio.to_raster("/data/res/test_merge_ifg.tif")

In [ ]:
# compute phase and coherence

# avoid metadata being lost in arithmetic opetations
xr.set_options(keep_attrs=True)
ifg = riox.open_rasterio("/data/res/test_merge_ifg.tif")
phi = np.arctan2(ifg[1], ifg[0])
img = np.sum(np.square(ifg),0)
phi.rio.to_raster("/data/res/test_merge_phi2.tif", nodata=0)
img.rio.to_raster("/data/res/test_merge_coh2.tif", nodata=0)

In [ ]:
import folium
m = Map()
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

# hack: avoid titiler caching the tiles
scale_phi = np.pi + np.random.rand() * 1e-6
rnd_coh = np.random.rand() * 1e-6

show_cog("/data/res/test_merge_coh2.tif", m,rescale=f"0,{1+rnd_coh}")
show_cog("/data/res/test_merge_phi2.tif", m,rescale=f"-{scale_phi}, {scale_phi}",colormap=palette_phi())
LayerControl().add_to(m)
m